In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter
import hvplot.xarray
import scipy.constants

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

import colorednoise as cn

#sys.path.append("../../preprocessing/")
#from generate_chirp import generate_chirp

sys.path.append("..")
from processing import pulse_compress

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 12})
formatter_hz = EngFormatter(unit='Hz')

In [ ]:
def generate_chirp_with_jitter(config, t_jitter=0):
    """
    Generate a chirp according to parameters in the config dictionary, typically
    loaded from a config YAML file.

    Returns a tuple (ts, chirp_complex), where ts is a numpy array of time
    samples, and chirp_complex is a numpy array of complex floating point values
    representing the chirp.

    If you're looking for a floating point valued chirp to use in convolution,
    this is probably the right function.

    This function does not convert the complex numpy array to the cpu format
    expected by the radar code. If you want to produce samples to feed the radar
    code, look at `generate_from_yaml_filename` (later in this file) instead.
    """
    # Load parameters
    gen_params = config["GENERATE"]
    chirp_type = gen_params["chirp_type"]
    sample_rate = gen_params["sample_rate"]
    chirp_bandwidth = gen_params["chirp_bandwidth"]
    offset = gen_params.get("lo_offset_sw", 0)
    window = gen_params["window"]
    chirp_length = gen_params["chirp_length"]
    pulse_length = gen_params.get("pulse_length", chirp_length) # default to chirp_length is no pulse_length is specified

    # Build chirp

    end_freq = chirp_bandwidth / 2 # Chirp goes from -BW/2 to BW/2
    start_freq = -1 * end_freq

    start_freq += offset
    end_freq += offset

    ts = np.arange(0, chirp_length-(1/(2*sample_rate)), 1/(sample_rate)) + t_jitter
    ts_zp = np.arange(0, (pulse_length)-(1/(2*sample_rate)), 1/(sample_rate))

    if chirp_type == 'linear':
        ph = 2*np.pi*((start_freq)*ts + (end_freq - start_freq) * ts**2 / (2*chirp_length))
    elif chirp_type == 'hyperbolic':
        ph = 2*np.pi*(-1*start_freq*end_freq*chirp_length/(end_freq-start_freq))*np.log(1- (end_freq-start_freq)*ts/(end_freq*chirp_length))
    else:
        ph = 2*np.pi*(start_freq*ts + (end_freq - start_freq) * ts**2 / (2*chirp_length))
        printf("[ERROR] Unrecognized chirp type '{chirp_type}'")
        return None, None

    chirp_complex = np.exp(1j*ph)

    if window == "blackman":
        chirp_complex = chirp_complex * np.blackman(chirp_complex.size)
    elif window == "hamming":
        chirp_complex = chirp_complex * np.hamming(chirp_complex.size)
    elif window == "kaiser14":
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 14.0)
    elif window == "kaiser10":
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 10.0)
    elif window == "kaiser18": 
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 18.0)
    elif window != "rectangular":
        print("[ERROR] Unrecognized window function '{window}'")
        return None, None

    chirp_complex = np.pad(chirp_complex, (int(np.floor(ts_zp.size - ts.size)/2),), 'constant')

    chirp_complex = chirp_complex

    return ts_zp, chirp_complex


In [ ]:
def plot_spectrum(ts, sig, sample_rate, one_sided=False, log_scale=False, nperseg=2**16, ax=None, label="", alpha=1.0, linestyle='-', time_delay=None):
    if ax is None:
        fig, ax = plt.subplots()
        fig.tight_layout()
    else:
        fig = None

    # Calculate power spectral density
    freq, spectrum = scipy.signal.welch(sig, fs=sample_rate, nperseg=nperseg, return_onesided=one_sided, detrend=False, window='rectangular', scaling='density')
    
    if time_delay is not None:
        ratio = 4 * (np.sin(np.pi * freq * time_delay))**2
    
    if not one_sided:
        freq = np.fft.fftshift(freq)
        spectrum = np.fft.fftshift(spectrum)
    
    ax.plot(freq, 10*np.log10(np.abs(spectrum)), label=label, alpha=alpha, linestyle=linestyle)

    if time_delay is not None:
        ax.plot(freq, 10*np.log10(np.abs(ratio*spectrum)), label=f"Theoretical for delay {time_delay}", alpha=alpha, linestyle='--')

    if log_scale:
        ax.set_xscale('log')
        ax.set_xlim([0.01, None])
        # if not one_sided:
        #     raise ValueError("Can't use log scale with two-sided spectrum")
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Power [dB]')
    ax.set_title('Frequency Domain')
    ax.grid(True)

    return fig, ax


In [ ]:
fs = 56e6
ts = np.arange(0, 1.0, 1/fs)

f_minus2 = 10**(-82/20) * cn.powerlaw_psd_gaussian(2, len(ts)) # -22
f_minus1 = 10**(-105/20) * cn.powerlaw_psd_gaussian(1, len(ts))
f_zero = 10**(-80/20) * cn.powerlaw_psd_gaussian(0, len(ts))
ph_noise_ref_osc = 10**(5/20) * (f_minus2 + f_minus1 + f_zero) # 55

ph_noise_clock = (fs/40e6) * ph_noise_ref_osc
ph_noise_lo = (400e6/40e6) * ph_noise_ref_osc

In [ ]:
start_freq, end_freq = -25e6, 25e6
chirp_length = 20e-6
chirp_length_samples = int(np.floor(chirp_length * fs))
chirp_length = chirp_length_samples / fs
reflection_distance = 3000 # m, one-way
reflection_time = 2*reflection_distance / (scipy.constants.speed_of_light * np.sqrt(3.17))
reflection_samples = int(np.floor(reflection_time * fs))
pri = 50e-6

fig, ax = plt.subplots(figsize=(8,6)) # 5,4

plot_spectrum(ts, ph_noise_ref_osc, fs, one_sided=True, log_scale=True, ax=ax, label="Reference Oscillator Phase Noise", nperseg=2**19, linestyle='--', alpha=0.5)
ax.scatter(np.array([0.0001, 0.001, 0.01, 0.1, 1, 5])*1e6, np.array([-117, -137, -149,-151,-151,-151])+3, color='black', marker='x', zorder=10, label="Reference")

# ADC+DAC phase noise
chirp_mask = (ts / pri) % 1 < (chirp_length / pri)
ts_chirps = ts[chirp_mask]
ts_chirps_rel = np.tile(ts[:chirp_length_samples], int(np.ceil(ts_chirps.size / chirp_length_samples)))[:ts_chirps.size]

def theta(t):
    t = t % pri
    t = np.maximum(np.minimum(t, chirp_length), 0)
    return 2*np.pi*(start_freq*t + (end_freq - start_freq) * t**2 / (2*chirp_length))

t_jitter = ph_noise_clock / (2*np.pi*fs)
t_jitter_interp = scipy.interpolate.interp1d(ts, t_jitter, kind='linear', fill_value=0, bounds_error=False)

ph_bb_actual = theta(ts_chirps_rel + t_jitter_interp(ts_chirps) - t_jitter_interp(ts_chirps + t_jitter_interp(ts_chirps) - reflection_time))
ph_no_noise = theta(ts_chirps_rel)

plot_spectrum(ts_chirps, ph_bb_actual - ph_no_noise, fs, one_sided=True, log_scale=True, ax=ax, label="ADC+DAC Phase Noise", nperseg=2**19)

#plot_spectrum(ts_chirps, (end_freq/fs)*ph_noise_clock[chirp_mask], fs, one_sided=True, log_scale=True, ax=ax, label="ADC+DAC Phase Noise (approx)", nperseg=2**17, time_delay=reflection_time)

# Mixers phase noise
plot_spectrum(ts[:-reflection_samples], ph_noise_lo[:-reflection_samples] - ph_noise_lo[reflection_samples:], fs, one_sided=True, log_scale=True, ax=ax, label="Mixer Phase Noise", nperseg=2**19)

#plot_spectrum(ts, ph_noise_lo, fs, one_sided=True, log_scale=True, ax=ax, label="LO Phase Noise", nperseg=2**19, time_delay=reflection_time)

ax.set_title(f'Phase Noise Spectra for {reflection_distance} m Reflector')

ax.set_xlim([100, 1e6])
ax.set_xticks([100, 1e3, 10e3, 100e3, 1e6])
ax.set_xticklabels([formatter_hz(x) for x in ax.get_xticks()])
ax.set_ylabel('Phase Noise Spectral Density\n[10 log10(rad^2/Hz)]')
ax.tick_params(axis='x', labelrotation=45)
ax.legend(loc="upper left")
fig.tight_layout()

In [ ]:
#fig.savefig(f"ph_noise_spectra_{reflection_distance}m_TMP.png", dpi=500)

In [ ]:
noise_std = 0.00

cl = chirp_length_samples

reference_chirp = np.exp(1j * theta(ts[:cl]))

lo_phase_diff = ph_noise_lo[:-reflection_samples] - ph_noise_lo[reflection_samples:]
chirp_phases = ph_bb_actual + lo_phase_diff[chirp_mask[:-reflection_samples]]
chirp_phases = chirp_phases[:cl * (len(chirp_phases)//cl)]
chirp_phases = np.reshape(chirp_phases, (-1, cl))
chirps = np.exp(1j*chirp_phases) + np.random.normal(0, noise_std, chirp_phases.shape) + 1j*np.random.normal(0, noise_std, chirp_phases.shape)
n_chirps, _ = chirps.shape
chirps.shape

In [ ]:
fig, ax = plt.subplots()
ax.plot(ts[:cl], chirp_phases[300,:])

In [ ]:
#stacked = np.mean(chirps[:1000,:], axis=0)

fast_time, compressed = pulse_compress(np.pad(chirps[500,:],300), reference_chirp, fs)

fig, ax = plt.subplots()
ax.plot(fast_time, 20*np.log10(np.abs(compressed)))
#ax.set_ylim([-6,2])
ax.grid()

In [ ]:
n_stacks = np.append(np.logspace(0, int(np.log10(n_chirps)), 10*(int(np.log10(n_chirps))+1), dtype=int), int(n_chirps))
peaks_lin_mean = np.zeros_like(n_stacks, dtype=np.float64)
peaks_lin_std = np.zeros_like(n_stacks, dtype=np.float64)
peak_phases_nstack_1 = np.zeros((n_chirps,))

for i, n_stack in enumerate(n_stacks):
    peaks_lin = []
    for start_idx in np.arange(0, n_chirps-n_stack+1, n_stack):
        stacked = np.mean(chirps[start_idx:start_idx+n_stack, :], axis=0)
        fast_time, compressed = pulse_compress(np.pad(stacked, 300), reference_chirp, fs)
        peaks_lin.append(np.max(np.abs(compressed)))
        if n_stack == 1:
            peak_phases_nstack_1[start_idx] = np.angle(compressed[np.argmax(np.abs(compressed))])
    
    peaks_lin_mean[i] = np.mean(peaks_lin)
    peaks_lin_std[i] = np.std(peaks_lin)

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))
plot_spectrum(np.arange(len(peak_phases_nstack_1)), np.unwrap(peak_phases_nstack_1), 1/pri, one_sided=True, log_scale=True, ax=ax, label="Phase of Peak", nperseg=2**19)
ax.set_title('Spectrum of peak phase\nafter pulse compression')
ax.set_xlim([1, 1e4])
fig.tight_layout()


In [ ]:
fig_power, ax_power = plt.subplots(figsize=(8,6))
first_plot = True

In [ ]:
effective_prf = 1/(n_stacks * pri)

# Peak power vs n_stack
if noise_std > 0:
    label = f"{reflection_distance} m w/ noise std {noise_std}"
else:
    label = f"{reflection_distance} m"
ax_power.scatter(n_stacks, 20*np.log10(peaks_lin_mean), label=label)
ax_power.set_xscale('log')

ax_power.set_xscale('log')
#ax.set_xlim(ax.get_xlim())
x_n = np.logspace(0, int(np.log10(n_chirps)), (int(np.log10(n_chirps))+1), dtype=int)
ax_power.set_xticks(x_n)
ax_power.set_xlabel("Number of stacked chirps")
ax_power.set_xlim([1, 1000])

if first_plot:
    ax_prf = ax_power.twiny()
    first_plot = False
ax_prf.set_xlabel("Effective Pulse Repetition Frequency")
ax_prf.set_xlim(1/(np.array(ax_power.get_xlim()) * pri))
ax_prf.set_xscale('log')
x_pri = 1/(x_n * pri)
ax_prf.set_xticks(x_pri)
ax_prf.set_xticklabels([f"{formatter_hz(x)}" for x in x_pri])
ax_prf.tick_params(axis='x', labelrotation=45)

ax_power.set_ylabel("Pulse Compressed Peak Power [dB]")

ax_power.grid('both')
ax_power.legend()

fig_power.tight_layout()

fig_power

In [ ]:
ax_power.legend(loc='upper right')
fig_power

In [ ]:
#fig_power.savefig("pc_peak_power_distance_TMP.png", dpi=500)